In [48]:
import json
from enum import Enum
from gurobipy import Model, GRB, quicksum
import numpy as np



In [49]:
class DATASET(Enum):
    TOY = "toy"
    MEDIUM = "medium"
    LARGE = "large"

In [50]:
def load_data(name):
    """name must be an instance of DATASET like DATASET.TOY for example"""
    if not isinstance(name, DATASET):
        raise TypeError("direction must be an instance of DATASET Enum")
    with open(f"../data/{name.value}_instance.json", "r") as f:
        data = json.load(f)
    return data


def get_dims(data):
    return (
        len(data["staff"]),
        data["horizon"],
        len(data["qualifications"]),
        len(data["jobs"]),
    )

In [51]:
def init_model():
    m = Model("Project modelling")
    return m

In [52]:
def create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs):
    X = model.addVars(n_staff, horizon, n_qualifs, n_jobs, vtype=GRB.BINARY, name="assignements")
    J = model.addVars(n_jobs, vtype=GRB.BINARY, name="completion")
    D = model.addVars(n_jobs, lb=0, ub=horizon + 1, vtype=GRB.INTEGER, name="end_dates")
    L = model.addVars(n_jobs, lb=0, ub=horizon + 1, vtype=GRB.INTEGER, name="nb_dates_late")
    return model, X, J, D, L


In [53]:
def add_constraints_for_J(model, X, J, jobs, qualifications):
    for index_job in range(len(J)):
        model.addGenConstrIndicator(
            J[index_job],
            True,
            (
                sum(X[:, :, index_k, index_job])
                >= jobs[index_job]["working_days_per_qualification"][k]
                for index_k, k in enumerate(qualifications)
            ),
        )
        model.addGenConstrIndicator(
            J[index_job],
            False,
            not (
                sum(X[:, :, index_k, index_job])
                >= jobs[index_job]["working_days_per_qualification"][k]
                for index_k, k in enumerate(qualifications)
            ),
        )


def add_constraints_for_D(model, X, J, D, horizon):
    for index_job in range(len(J)):

        start_date = min([j for j in range(horizon) if sum(X[:, j, :, index_job]) >= 0])
        end_date = max([j for j in range(horizon) if sum(X[:, j, :, index_job]) >= 0])
        range = end_date - start_date + 1

        model.addGenConstrIndicator(
            J[index_job],
            True,
            D[index_job, :] == [start_date, end_date, range],
        )
        model.addGenConstrIndicator(J[index_job], False, D[index_job, :] == [0, horizon + 1, 0])

In [61]:
def add_constraints_end_dates(model, X, D, jobs, qualifications, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        X[i, j, k, l] * j <= D[l] 
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
    )

In [55]:
def add_constraints_lateness(model, D, L, jobs, n_staff, horizon, n_qualifs, n_jobs):
        model.addConstrs(
            D[l] - jobs[l]["due_date"] <= L[l] for l in range(n_jobs)
        )

In [56]:
def in_qualification(i, k, data):
    data = {l : data["staff"][l] for l in range(len(data["staff"]))}
    data = data[i]["qualifications"]
    return k in data


def add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if not in_qualification(i, k, data)
    )


def in_vacation(i, j, data):
    data = {l: data["staff"][l] for l in range(len(data["staff"]))}
    data = data[i]["vacations"]
    return j in data


def add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if in_vacation(i, j, data)
    )

In [57]:
def min_nb_projet_per_staff(model, n_staff, horizon, n_qualifs, n_jobs):
    obj1 = quicksum(M(i, j, k, l) for j in horizon for k in n_qualifs)
    obj1 = quicksum()

In [58]:
def add_profit_as_first_objective(model, J, D, jobs):

    benef = sum(
        [
            J[index_job] * (job.gain - job.daily_penalty * max(D[index_job, 1] - job.due_date, 0))
            for index_job, job in enumerate(jobs)
        ]
    )

    model.setObjective(benef, GRB.MAXIMIZE)

In [59]:
def main():
    # Importing data
    data = load_data(DATASET.TOY)
    n_staff, horizon, n_qualifs, n_jobs = get_dims(data)

    # Instanciation du modèle
    model = init_model()

    # Création des variables : binaires dans X et J, entières de 0 à horizon + 3
    model, X, J, D, L = create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs)

    # maj du modèle
    model.update()

    # Ajout des 3 constraintes
    add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)
    add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)
    add_constraints_end_dates(model, X, D, data["jobs"], data["qualifications"], n_staff, horizon, n_qualifs, n_jobs)
    add_constraints_for_D(model, X, J, D, horizon)

    # Fonction Objectif
    add_profit_as_first_objective(model, J, D, data["jobs"])

    # Paramétrage (mode mute)
    # model.params.outputflag = 0

    # Résolution du PL
    model.optimize()

In [62]:
main()

UnboundLocalError: local variable 'range' referenced before assignment